# Trabalho computacional, Codigo

Nome: Maverick Alekyne de Sousa Ribeiro - 541062
Nome: Ana Livia Sousa Davi Taveira - 536158

Professor: Hitalo Joseferson



Informações adicionais:

Realizar testes com 100, 200, 300 e 500 formigas formigas;

Utilize α = 1 e β = 5;

Número de iterações: 1000;
Número de locais: 35;



No trabalho computacional utilizamos somente uma matriz de 5 caminhos, pois 35 era muito, e pra compreensão e vizualização dos resultados foi melhor deixar 5 caminhos



In [19]:
import numpy as np
from numpy.random import choice as np_choice

class AntColony:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):  # Corrigido o nome do construtor
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        all_time_shortest_path = None
        all_time_shortest_distance = np.inf

        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheromone(all_paths, self.n_best)
            shortest_path = min(all_paths, key=lambda x: x[1])

            print(f"Iteração {i + 1}: {shortest_path}")

            if all_time_shortest_path is None or shortest_path[1] < all_time_shortest_distance:
                all_time_shortest_path = shortest_path[0]
                all_time_shortest_distance = shortest_path[1]

            self.pheromone *= self.decay  # Decay pheromone after each iteration

        return all_time_shortest_path, all_time_shortest_distance

    def spread_pheromone(self, all_paths, n_best):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for i in range(len(path) - 1):
                move = (path[i], path[i + 1])
                self.pheromone[move] += 1.0 / dist

            move = (path[-1], path[0])
            self.pheromone[move] += 1.0 / dist

    def gen_path_dist(self, path):
        total_dist = 0
        for i in range(len(path) - 1):
            total_dist += self.distances[path[i], path[i + 1]]
        total_dist += self.distances[path[-1], path[0]]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for _ in range(self.n_ants):
            path = self.gen_path(0)
            dist = self.gen_path_dist(path)
            all_paths.append((path, dist))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start

        for _ in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append(move)
            prev = move
            visited.add(move)

        path.append(start)
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0
        row = pheromone ** self.alpha * ((1.0 / dist) ** self.beta)
        row_sum = row.sum()

        if row_sum == 0:
            return np.random.choice(self.all_inds)

        norm_row = row / row_sum
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move


# Definindo o número de locais
n_locais = 35

# Criando uma matriz de distâncias com valores aleatórios entre 1 e 100 e mudando o tipo para float
distances = np.random.randint(1, 101, size=(n_locais, n_locais)).astype(float)

# Definindo a diagonal principal como um valor grande (mas não infinito)
np.fill_diagonal(distances, 1000)  # Definindo um valor suficientemente grande para evitar auto-loops

# Tornando a matriz simétrica (garantindo que todos os locais estejam conectados)
distances = np.minimum(distances, distances.T)

# Parâmetros do algoritmo (constantes)
n_best = 3
n_iterations = 1000
decay = 0.5
alpha = 1
beta = 5

# Testando diferentes quantidades de formigas
for n_ants in [100, 200, 300, 500]:
    print(f"\nExecutando o ACO com {n_ants} formigas:\n")
    ant_colony = AntColony(distances, n_ants, n_best, n_iterations, decay, alpha, beta)
    best_path, best_distance = ant_colony.run()
    print(f"Melhor caminho com {n_ants} formigas: {best_path}")
    print(f"Melhor distância com {n_ants} formigas: {best_distance}")



Executando o ACO com 100 formigas:

Iteração 1: ([9, 34, 22, 7, 11, 13, 27, 8, 4, 2, 12, 3, 6, 19, 20, 23, 21, 32, 16, 25, 26, 33, 10, 18, 31, 17, 30, 14, 28, 29, 1, 15, 5, 24, 0], 171.0)
Iteração 2: ([9, 34, 16, 25, 26, 27, 13, 28, 14, 31, 17, 30, 11, 7, 22, 23, 21, 32, 12, 2, 18, 10, 33, 20, 19, 6, 3, 8, 4, 5, 24, 15, 29, 1, 0], 162.0)
Iteração 3: ([9, 34, 22, 27, 13, 28, 14, 31, 17, 30, 33, 10, 18, 2, 12, 32, 21, 23, 20, 19, 6, 3, 8, 4, 5, 24, 16, 25, 26, 7, 11, 15, 29, 1, 0], 154.0)
Iteração 4: ([9, 34, 16, 25, 26, 27, 13, 28, 14, 31, 17, 30, 33, 10, 18, 2, 12, 32, 21, 23, 20, 19, 6, 3, 8, 4, 5, 24, 7, 11, 15, 29, 1, 22, 0], 147.0)
Iteração 5: ([9, 34, 16, 25, 26, 27, 13, 28, 14, 31, 17, 30, 33, 10, 18, 2, 12, 32, 21, 23, 20, 19, 6, 3, 8, 4, 5, 24, 7, 11, 15, 29, 1, 22, 0], 147.0)
Iteração 6: ([9, 34, 16, 25, 26, 27, 13, 28, 14, 1, 29, 15, 31, 17, 30, 11, 7, 22, 23, 21, 32, 12, 2, 18, 10, 33, 20, 19, 6, 3, 8, 4, 5, 24, 0], 128.0)
Iteração 7: ([9, 34, 16, 25, 26, 27, 13, 28, 14, 1,